In [45]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from time import time
import pathlib
import imageio
from PIL import Image
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.utils import to_categorical

In [47]:
# Define the device (use GPU if available)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
#device = 'cpu'
print(f'Using device: {device}')

Using device: cuda


In [49]:
import os
from PIL import Image
import pandas as pd
 
def load_png_files_to_df(directory):
    # List to hold the image data
    image_data = []
 
    # Iterate through files in the directory
    for filename in os.listdir(directory):
        if filename.endswith('.png'):
            file_path = os.path.join(directory, filename)
            try:
                # Open the image file
                img = Image.open(file_path)
                image_data.append({'filename': filename, 'image': img})
            except Exception as e:
                print(f"Error opening {filename}: {e}")
 
    # Create a DataFrame from the image data
    df = pd.DataFrame(image_data)
    return df
 
# Example usage
directory = 'faces/kawamura'
df_images = load_png_files_to_df(directory)
 
# Display the DataFrame
print(df_images)

                                filename  \
0           kawamura_left_angry_open.png   
1         kawamura_left_angry_open_2.png   
2         kawamura_left_angry_open_4.png   
3     kawamura_left_angry_sunglasses.png   
4   kawamura_left_angry_sunglasses_2.png   
..                                   ...   
88            kawamura_up_sad_open_2.png   
89            kawamura_up_sad_open_4.png   
90        kawamura_up_sad_sunglasses.png   
91      kawamura_up_sad_sunglasses_2.png   
92      kawamura_up_sad_sunglasses_4.png   

                                                image  
0   <PIL.PngImagePlugin.PngImageFile image mode=L ...  
1   <PIL.PngImagePlugin.PngImageFile image mode=L ...  
2   <PIL.PngImagePlugin.PngImageFile image mode=L ...  
3   <PIL.PngImagePlugin.PngImageFile image mode=L ...  
4   <PIL.PngImagePlugin.PngImageFile image mode=L ...  
..                                                ...  
88  <PIL.PngImagePlugin.PngImageFile image mode=L ...  
89  <PIL.PngImagePlugin

In [71]:
labels = 'ttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttt'
X_train, X_test, y_train, y_test = train_test_split(df_images, labels, test_size=0.2, random_state=42)
 
# Convert labels to one-hot encoding
y_train = to_categorical(y_train, num_classes=2)
y_test = to_categorical(y_test, num_classes=2)
 
# Create and train the model
model = create_model(input_shape=(64, 64, 3))
model.fit(X_train, y_train, epochs=5, batch_size=32, validation_data=(X_test, y_test))
 
# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Test Accuracy: {accuracy * 100:.2f}%')

ValueError: Found input variables with inconsistent numbers of samples: [93, 94]

In [ ]:
def create_model(input_shape):
    model = Sequential([
        Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),
        MaxPooling2D((2, 2)),
        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        Flatten(),
        Dense(64, activation='relu'),
        Dense(2, activation='softmax')  # Binary classification (bias or not)
    ])
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model